In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
# 데이터 읽어드리기

train.head()
# 데이터의 형태를 참고해본다.

train.loc[train[np.isnan(train['Age'])].index, "Age"] = round(train["Age"].mean(), 0)
train["Embarked"][train["Embarked"].isnull()] = "S"
# Age에 대한 결측치는 전체 나이의 평균을 대입했고
# Embarked에 대한 결측치는 S로 처리했다.

del train[train.columns[0]] # passenger_id 처리
del train["Ticket"]
del train["Name"]
del train["Cabin"]
train["familySize"] = train["SibSp"] + train["Parch"]
del train["SibSp"]
del train["Parch"]
train["travelAlone"] = np.where(train["familySize"]>0, 0,1)
train.loc[train.loc[train['Sex']=='male'].index, "Sex"] = 0
train.loc[train.loc[train['Sex']=='female'].index, "Sex"] = 1
train.loc[train.loc[train['Embarked']=='S'].index, "Embarked"] = 0
train.loc[train.loc[train['Embarked']=='C'].index, "Embarked"] = 1
train.loc[train.loc[train['Embarked']=='Q'].index, "Embarked"] = 2

# Ticket, Name, Cabin 은 불필요한 데이터로 하여 제거하였다.
# SibSp 와 Parch를 이용해 familySize를 만들고
# familySize 가 0일 경우 travelAlone 라는 변수를 추가하여 1 을 대입했다.
# Sex, Embarked에 대한 데이터를 학습시키기 편하게 전처리하였다.

train

target = train.loc[:,"Survived"]
target =  np.asarray(target, dtype = "int")
values = train.drop(["Survived"], axis = 1)

# Survived를 target 변수로 뽑아낸다.

values.isnull().sum()

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=10)
gn = GaussianNB()

list_ = []

# fit에는 독립변수들은 dataframe 형태로 
# 종속변수는 numpy.ndarray 형태로 들어간다.
for train_index, test_index in kf.split(values):
    target_train, target_test = target[train_index], target[test_index]
    values_train, values_test = values.iloc[train_index], values.iloc[test_index]
    nb_model = gn.fit(values_train, target_train)
    list_ = np.append(list_,cross_val_score(nb_model, values_test, target_test).mean())

round(list_.mean()*100,3)
# 총평균

